In [ ]:
import pandas as pd
import os
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np

https://tccon-wiki.caltech.edu/Main/GGG2020DataChanges

# From results akall

In [ ]:
results_path = '/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/results/daily'
ak_dates = ['20220616','20220910','20221122','20230131','20230428','20230710','20231006']
specs = {'ch4':[5938,6002,6076],'co2':[6220,6339],'co':[4233,4290]}

all_dfs = {}
gas_dfs = {spec: {} for spec in specs.keys()}

for ak_date in ak_dates:
    all_dfs[ak_date] = {}
    day_path = os.path.join(results_path, ak_date)
    akall_files = [file for file in os.listdir(day_path) if (file.endswith('akall')) & (file.split('_')[0] in specs)]
    for spec in specs.keys():
        files = [file for file in akall_files if file.startswith(f'{spec}_')]
        spec_df = pd.DataFrame()
        for file in files:
            window = file.split('_')[1].split('.')[0]
            df = pd.read_csv(os.path.join(day_path, file), header=1, sep='\s+')
            first_df = df.groupby('ispec').first()[['sza', 'ak', 'p']]
            for col in first_df.columns:
                first_df = first_df.rename(columns={col: f'{col}_{window}'})
            spec_df = pd.concat([spec_df, first_df], axis=1)
        avg_cols = [col for col in spec_df.columns if 'ak' in col]
        spec_df['avg_ak'] = spec_df.apply(lambda row: row[avg_cols].mean(), axis=1)
        all_dfs[ak_date][spec] = spec_df
        gas_dfs[spec][ak_date] = spec_df 


In [ ]:
gas_dfs['co']['20220616']

In [ ]:
bin_dfs = {}
for spec in specs.keys():
    spec_df = pd.DataFrame()
    for date in all_dfs.keys():
        window = specs[spec][0]
        df = all_dfs[date][spec]
        df.index = round(df[f'sza_{window}']*2)/2
        df_binned = df.groupby(df.index).mean()[['avg_ak']]
        df_binned = df_binned.rename(columns = {'avg_ak':f'avg_ak_{date}'})
        spec_df = pd.concat([spec_df,df_binned],axis = 1)
    bin_dfs[spec] = spec_df.mean(axis = 1)

In [ ]:
spec = 'co2'
date = list(all_dfs.keys())[0]
for date in all_dfs.keys():
    df = all_dfs[date][spec]
    window = specs[spec][0]
    # for window in windows:
    #     plt.plot(df[f'sza_{window}'],df[f'ak_{window}'],label = date)

    plt.plot(df[f'sza_{window}'],df['avg_ak'],label=date)

plt.plot(bin_dfs[spec],color = 'k',linewidth = 5)

plt.legend()


In [ ]:
# # SAVING --careful not to overwrite
import pickle
pickle_path = f'/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/Pickled_files/averaging_kernel'

fname = 'mean_sza0.5.pkl'
with open(os.path.join(pickle_path,fname), 'wb') as handle:
    pickle.dump(bin_dfs, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
gas = 'co'
date = '20220616'

In [ ]:
gas_df = gas_dfs[gas][date]
fig, ax = plt.subplots()
for window in specs[gas]:
    ax.plot(gas_df[f'sza_{window}'],gas_df[f'ak_{window}'],label = f'{window} cm-1')
ax.legend()
ax.set_title(f'{date}')
ax.set_xlabel('SZA')
ax.set_ylabel(f'{gas} Surface AK')
plt.show()

In [ ]:
gas_df = gas_dfs[gas][date]
fig, ax = plt.subplots()
for window in specs[gas]:
    ax.plot(gas_df.index,gas_df[f'ak_{window}'],label = f'{window} cm-1')
ax.legend()
ax.set_title(f'{gas} AKs for {date}')
ax.set_xlabel('ispec (~time of day)')
ax.set_ylabel(f'{gas} surface AK')
plt.show()

In [ ]:
window = 6076

fig, ax = plt.subplots()
for date,df in gas_dfs[gas].items():
    ax.plot(df[f'sza_{window}'],df[f'ak_{window}'],label = f'{date}')
    ax.legend()
    ax.set_title(f'Window {window} cm-1')
    ax.set_xlabel('SZA')
    ax.set_ylabel(f'{gas} Surface AK')
plt.show()

# With NC Files

In [ ]:
ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/results/daily/20220616/ha20220616_20220617.private.nc')

In [ ]:
airmass = ds['o2_7885_am_o2']
xgas_name ='xco'
xgas = ds[xgas_name]
slant_xgas = airmass * xgas

In [ ]:
slant_xgas_values.max()

In [ ]:
# Prepare the output array
interpolated_values = []

# Extract required variables
slant_xgas_values = slant_xgas.values  # Slant XCO2 values for each time
ak_slant_xgas_bin = ds[f'ak_slant_{xgas_name}_bin'].values  # Slant XCO2 bin edges
ak_xgas = ds[f'ak_{xgas_name}'].values  # AK table, shape (ak_altitude, ak_slant_xgas_bin)

# Loop through each time point
for i, slant_value in enumerate(slant_xgas_values):
    # Interpolate for each altitude
    interpolated_ak = np.empty(ak_xgas.shape[0])  # Placeholder for interpolated values at this time
    
    for j in range(ak_xgas.shape[0]):  # Iterate over altitude
        interpolated_ak[j] = np.interp(
            slant_value,                   # Current slant XCO2 value
            ak_slant_xgas_bin,            # Slant XCO2 bins
            ak_xgas[j, :]                 # AK values at this altitude
        )
    
    interpolated_values.append(interpolated_ak)

# Convert results to xarray DataArray
ak_interp = xr.DataArray(
    data=np.array(interpolated_values),  # Stack the results into a 2D array
    dims=["time", "ak_altitude"],        # Dimensions are time and altitude
    coords={
        "time": ds["time"],              # Time coordinate from the original dataset
        "ak_altitude": ds["ak_altitude"] # Altitude coordinate from the original dataset
    },
    name="ak_interp"                     # Name of the new DataArray
)


In [ ]:
ak_interp

In [ ]:
ak_interp.isel(ak_altitude = 0).plot()

## TCCON Check

In [ ]:
public_ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/lin-group9/agm/TCCON/ci20120920_20240421.public.qc.nc')
public_ds = public_ds.where((public_ds.time > pd.to_datetime('2023-05-02'))&(public_ds.time < pd.to_datetime('2023-07-01')),drop = True)
private_ds = xr.open_dataset('/uufs/chpc.utah.edu/common/home/u0890904/LAIR_1/Data/TCCON/ci20230502_20230701.private.nc')

In [ ]:
public_co2_ak = public_ds['ak_xco2'].isel(ak_altitude = 0)

In [ ]:
airmass = private_ds['o2_7885_am_o2']
xco2 = private_ds['xco2']
slant_xco2 = airmass * xco2

# # Prepare the output array
interpolated_values = []

# Extract required variables
slant_xco2_values = slant_xco2.values  # Slant XCO2 values for each time
ak_slant_xco2_bin = private_ds['ak_slant_xco2_bin'].values  # Slant XCO2 bin edges
ak_xco2 = private_ds['ak_xco2'].values  # AK table, shape (ak_altitude, ak_slant_xgas_bin)

# Loop through each time point
for i, slant_value in enumerate(slant_xco2_values):
    # Interpolate for each altitude
    interpolated_ak = np.empty(ak_xco2.shape[0])  # Placeholder for interpolated values at this time
    
    for j in range(ak_xco2.shape[0]):  # Iterate over altitude
        interpolated_ak[j] = np.interp(
            slant_value,                   # Current slant XCO2 value
            ak_slant_xco2_bin,            # Slant XCO2 bins
            ak_xco2[j, :]                 # AK values at this altitude
        )
    
    interpolated_values.append(interpolated_ak)

# Convert results to xarray DataArray
ak_interp = xr.DataArray(
    data=np.array(interpolated_values),  # Stack the results into a 2D array
    dims=["time", "ak_altitude"],        # Dimensions are time and altitude
    coords={
        "time": private_ds["time"],              # Time coordinate from the original dataset
        "ak_altitude": private_ds["ak_altitude"] # Altitude coordinate from the original dataset
    },
    name="ak_interp"                     # Name of the new DataArray
)

private_co2_ak = ak_interp.isel(ak_altitude = 0)

In [ ]:
fig = make_subplots(rows = 1, cols = 1)
fig.add_trace(go.Scatter(
    x = public_ds['time'],
    y = public_co2_ak,
    mode = 'markers',
    name = 'Public AK')
    ,row = 1,col = 1)
fig.add_trace(go.Scatter(
    x = private_ds['time'],
    y = private_co2_ak,
    mode = 'markers',
    name = 'Private AK')
    ,row = 1,col = 1)


# Checking for nc

In [ ]:
results_path = '/uufs/chpc.utah.edu/common/home/lin-group9/agm/EM27/ha/results/daily'
for date in os.listdir(results_path):
    if not any(file.endswith('.nc') for file in os.listdir(os.path.join(results_path, date))):
        print(date)